## Plot sfincs_map.nc results

We use the [cartopy](https://scitools.org.uk/cartopy/docs/latest/) package to plot maps. This packages provides a simple interface to plot geographic data and add background satellite imagery.

In [ ]:
# import dependencies
import xarray as xr
import numpy as  np
from os.path import join
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import hydromt

### Read model results

The model results in sfincs_map.nc are saved as in a staggered grid format, see [SGRID convention](https://publicwiki.deltares.nl/display/NETCDF/Deltares+proposal+for+Staggered+Grid+data+model+(SGRID)). Here we show how to retrieve the face values and translate the dimensions from node indices (m, n) to (x, y) coordinates in order to plot the results on a map.

In [ ]:
sfincs_root = 'venice_200' # (relative) path to sfincs root
mod = hydromt.SfincsModel(sfincs_root, mode='r')

In [ ]:
# read results 
ds_map = xr.open_dataset(join(mod.root, 'sfincs_map.nc'), chunks={'time':100})

# get values which are saved on cell faces
face_vars = [v for v in ds_map.data_vars if 'm' in ds_map[v].dims]
face_coords = {
    'x': xr.IndexVariable('x', ds_map['x'].isel(n=0).values),
    'y': xr.IndexVariable('y', ds_map['y'].isel(m=0).values)
}
encoding = {v :{'zlib': True} for v in face_vars}
ds_face = ds_map[face_vars].drop(['x', 'y']).rename({'n':'y', 'm':'x'}).assign_coords(face_coords).transpose('time', 'y', 'x', ...)
# set coordinate reference system based on epsg code in sfincs.inp > saved to spatial_ref coord
ds_face.raster.set_crs(mod.crs)
# inspect outcome
ds_face

### Calculate hmax and mask with surface water mask

First we have to mask the water depth based on a map of permanent water which we calculate from the Global Surface Water Occurence (GSWO) dataset. 

In [ ]:
# read global surface water occurance (GSWO) data to mask permanent water
print(mod.data_catalog['gswo'])
gswo = mod.data_catalog.get_rasterdataset('gswo', buffer=10)
# permanent water where water occurence > 5%
gswo_mask = gswo.raster.reproject_like(mod.staticmaps, method='max') <= 5

In [ ]:
hmin = 0.2 # minimum flood depth [m] to plot 
da_hmax = ds_face['h'].max('time') # get the maximum flood depth
# get overland flood depth with GSWO and set minimum flood depth
da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax>hmin)
# update attributes for colorbar label later
da_hmax_fld.attrs.update(long_name='flood depth', unit='m') 

### Plot maximum water depth

Here we plot the maximum water depth on top of the :py:meth:`~hydromt_sfincs.sfincs.SfincsModel.plot_basemaps` method.

In [ ]:
# create hmax plot and save to mod.root/figs/hmax.png
fig, ax = mod.plot_basemap(fn_out=None, variable=None, bmap='sat', geoms=['src', 'bnd'], figsize=(11,8))
# plot overland flooding based on gswo mask and mimum flood depth
cbar_kwargs = dict(aspect=20, fraction=.1, shrink=.7)
cax_fld = da_hmax_fld.plot(ax=ax, vmin=0, vmax=1.0, cmap=plt.cm.viridis, cbar_kwargs=cbar_kwargs)
# plot open water waterleves in purpule based on inverse gswo mask
cax_riv = da_zmax_sw.plot(ax=ax, vmin=-1, vmax=1.0, cmap=plt.cm.Blues, cbar_kwargs=cbar_kwargs)

ax.set_title(f'SFINCS maximum water depth')
# plt.savefig(join(mod.root, 'figs', 'hmax.png'), dpi=225, bbox_inches="tight")

### Animate water level

An animation is also simple to make with `matplotlib.animation` method. Here we add the surface water level in blue colors next to the overland flood depth with viridis colormap. 

In [ ]:
# differentiate h inside and outside permanent water mask
da_h = ds_face['h'].where(gswo_mask)
da_h = da_h.where(da_h>hmin).drop('spatial_ref')
da_h.attrs.update(long_name='flood depth', unit='m')
da_zs = ds_face['zs'].where(~gswo_mask).drop('spatial_ref')
da_zs.attrs.update(long_name='surface water level', unit='m+MSL')

In [ ]:
# create hmax plot and save to mod.root/figs/sfincs_h.mp4
# requires ffmpeg install with "conda install ffmpeg -c conda-forge"
from matplotlib import animation
step = 1 # one frame every <step> hr

def update_plot(i, da_h, cax_h, da_zs=None, cax_sw=None):
    da_h = da_h.isel(time=i)
    t = da_h.time.dt.strftime("%d-%B-%Y %H:%M:%S").item()
    ax.set_title(f'SFINCS water depth {t}')
    cax_h.set_array(da_h.values)
    if da_zs is not None and cax_zs is not None:
        da_zs= da_zs.isel(time=i)
        cax_zs.set_array(da_zs.values)

fig, ax = mod.plot_basemap(fn_out=None, variable=None, bmap='sat', geoms=['src', 'bnd'], figsize=(11, 8))
cax_h = da_h.isel(time=0).plot(ax=ax, vmin=0, vmax=1.0, cmap=plt.cm.viridis, cbar_kwargs=cbar_kwargs)
cax_zs = da_zs.isel(time=0).plot(ax=ax, vmin=-1, vmax=1, cmap=plt.cm.Blues, cbar_kwargs=cbar_kwargs)
plt.close()  # to prevent double plot

ani = animation.FuncAnimation(
    fig,
    update_plot,
    frames=np.arange(0, da_h.time.size, step),
    interval=250,     # ms between frames
    fargs=(da_h, cax_h, da_zs, cax_zs)
)

# to save to mp4
# ani.save(join(mod.root, 'figs', 'sfincs_h.mp4'), fps=4, dpi=200)

# to show in notebook:
from IPython.display import HTML
HTML(ani.to_html5_video())